# 2. Develop your segmentation model

In [ ]:
#Import libraries
import os
from model_class import ModelSegmentation
from aux_functions import slicing
import cv2
#Inputs
working_directory=r"C:\Users\Pheno\Documents\database_almondcv2\apple_CV"
pictures_directory=os.path.join(working_directory, "pruebas")
pre_model=os.path.join(working_directory, "models/yolo11s-seg.pt")
model_path=os.path.join(working_directory, "last.pt")
output_directory=os.path.join(working_directory,"output_directory")

### Slicing

In [ ]:
#Slice_pictures for training
slicing(input_folder=pictures_directory,output_directory=working_directory,name_slicing="Slices_apple",
         number_pictures=40, slice_height=640, slice_width=640)

### Training

In [ ]:
#Label with CVAT

zip_file_shell=os.path.join(working_directory,"apple_640.zip")

In [ ]:
#Model segmentation training

model=ModelSegmentation(working_directory=working_directory)
model.train_segmentation_model(input_zip=zip_file_shell, epochs=100,imgsz=640, name_segmentation="apple",
                                      pre_model=pre_model, batch=16)

### Deploy and reconstruct a picture

#### Slice_predict_reconstruct approach

In [13]:

model=ModelSegmentation(working_directory=working_directory)
masks=model.slice_predict_reconstruct(input_folder=pictures_directory,imgsz=640, model_path=model_path,
                                          slice_height=640, slice_width=640,overlap_height_ratio=0.2,
                                          overlap_width_ratio=0.2, conf=0.2)

Detected GPU: NVIDIA GeForce RTX 3060
Total GPU Memory: 12.00 GB


04/27/2025 11:11:02 - INFO - sahi.slicing -   image.shape: (1296, 1080)
04/27/2025 11:11:02 - INFO - sahi.slicing -   Num slices: 6 slice_height: 640 slice_width: 640


Processing Image 1/6


04/27/2025 11:11:02 - INFO - sahi.slicing -   image.shape: (1296, 1080)
04/27/2025 11:11:02 - INFO - sahi.slicing -   Num slices: 6 slice_height: 640 slice_width: 640
04/27/2025 11:11:02 - INFO - sahi.slicing -   image.shape: (1296, 1080)
04/27/2025 11:11:02 - INFO - sahi.slicing -   Num slices: 6 slice_height: 640 slice_width: 640


Processing Image 2/6
Processing Image 3/6
Processing Image 4/6


04/27/2025 11:11:03 - INFO - sahi.slicing -   image.shape: (1296, 1080)
04/27/2025 11:11:03 - INFO - sahi.slicing -   Num slices: 6 slice_height: 640 slice_width: 640
04/27/2025 11:11:03 - INFO - sahi.slicing -   image.shape: (1296, 1080)
04/27/2025 11:11:03 - INFO - sahi.slicing -   Num slices: 6 slice_height: 640 slice_width: 640
04/27/2025 11:11:03 - INFO - sahi.slicing -   image.shape: (1296, 1080)
04/27/2025 11:11:03 - INFO - sahi.slicing -   Num slices: 6 slice_height: 640 slice_width: 640


Processing Image 5/6
Processing Image 6/6


In [14]:
# To show the masks
os.makedirs(output_directory, exist_ok=True)
for mask in masks:
    cv2.imwrite(f"{output_directory}/{os.path.basename(mask[1])}", mask[0])


#### SAHI

In [ ]:
model=ModelSegmentation(working_directory=working_directory)
masks=model.predict_model_sahi(model_path=model_path, check_result=False, folder_input=pictures_directory,
                                            retina_masks=True,
                                              postprocess_match_threshold=0.2, overlap_height_ratio=0.2,
                                                overlap_width_ratio=0.2, postprocess_match_metric="IOS", 
                                                postprocess_type="GREEDYNMM", slice_height=640, slice_width=640,
                                                  confidence_treshold=0.8,
                                                  imgsz=640)

In [ ]:
# To show the masks
os.makedirs(output_directory, exist_ok=True)
for mask in masks:
    mask[0].export_visuals(export_dir=output_directory, hide_labels=True, rect_th=1, file_name=f"{os.path.basename(mask[1])}")